In [ ]:
import pandas as pd
import os

In [ ]:
file_paths = {
    "VDJdb_beta": vdjdb_beta_read_path,
    "McPAS_beta": mcpastcr_beta_read_path,
    "IEDB_beta": iedb_beta_read_path,
    "pMTnet_beta": pmtnet_beta_read_path,
    "VDJdb_paired": vdjdb_paired_read_path,
    "McPAS_paired": mcpastcr_paired_read_path,
    "IEDB_paired": iedb_paired_read_path
}

In [ ]:
# Alle Daten speichern
all_data = {}

# Daten einlesen und vorbereiten
for file_name, path in file_paths.items():
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, sep=None, engine="python")

            # tcr_key erstellen
            if "TRA_CDR3" in df.columns:
                df["tcr_key"] = df["TRA_CDR3"].astype(str) + '_' + df["TRB_CDR3"]
            else:
                df["tcr_key"] = df["TRB_CDR3"]

            all_data[file_name] = df[["tcr_key", "Epitope"]].drop_duplicates()

            print(f"{file_name} geladen mit {len(df)} Einträgen.")

        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
    else:
        print(f"Datei nicht gefunden: {file_name}")

# Analyse: Welcher Datensatz sollte im Testset sein?
for test_file_name, test_df in all_data.items():
    # Alle anderen Daten als Trainingsset
    train_df = pd.concat([data for name, data in all_data.items() if name != test_file_name]).drop_duplicates()

    # Erstelle Sets für schnelles Lookup
    seen_tcrs = set(train_df["tcr_key"])
    seen_epitopes = set(train_df["Epitope"])

    # TPP3-Definition: Weder TCR noch Epitope wurden im Trainingsset gesehen
    tpp3_pairs = test_df[~test_df["tcr_key"].isin(seen_tcrs) & ~test_df["Epitope"].isin(seen_epitopes)]

    # Ausgabe der Ergebnisse
    print(f"\n**Wenn {test_file_name} als Testset verwendet wird:**")
    print(f"  - TPP3-Paare im Testset: {len(tpp3_pairs)}")
    print(f"  - Gesamt Test-Paare: {len(test_df)}")

In [ ]:
import pandas as pd
import os

# Function to classify tasks based on TCR and epitope presence
def calculate_task(row, known_epitopes, known_tcr, paired=False):
    if paired:
        tra_cdr3 = str(row['TRA_CDR3']) if pd.notna(row['TRA_CDR3']) else ''
        trb_cdr3 = str(row['TRB_CDR3']) if pd.notna(row['TRB_CDR3']) else ''
        tcr = tra_cdr3 + '_' + trb_cdr3
    else:
        tcr = row['TRB_CDR3']
    
    epitope_exists = row['Epitope'] in known_epitopes
    cdr3_exists = tcr in known_tcr
    
    if epitope_exists and cdr3_exists:
        return 'TPP1'
    elif epitope_exists and not cdr3_exists:
        return 'TPP2'
    elif not epitope_exists and not cdr3_exists:
        return 'TPP3'
    elif not epitope_exists and cdr3_exists:
        return 'TPP4'
    raise Exception("Something seems wrong")

# Placeholder for the data
all_data = {}

# Load and prepare data
for file_name, path in file_paths.items():
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, sep=None, engine="python")

            # Create tcr_key
            if "TRA_CDR3" in df.columns:
                paired = True
                df["tcr_key"] = df["TRA_CDR3"].astype(str) + '_' + df["TRB_CDR3"]
            else:
                paired = False
                df["tcr_key"] = df["TRB_CDR3"]

            all_data[file_name] = df
            print(f"{file_name} geladen mit {len(df)} Einträgen.")

        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
    else:
        print(f"Datei nicht gefunden: {file_name}")

# Analyse: Classify TPP tasks
for test_file_name, test_df in all_data.items():
    # Define training data (excluding the current test set)
    train_df = pd.concat([data for name, data in all_data.items() if name != test_file_name]).drop_duplicates()

    seen_tcrs = set(train_df["tcr_key"])
    seen_epitopes = set(train_df["Epitope"])

    # Determine if it's a paired dataset
    paired = "TRA_CDR3" in test_df.columns

    # Apply classification
    test_df['task'] = test_df.apply(lambda row: calculate_task(row, seen_epitopes, seen_tcrs, paired=paired), axis=1)

    # Count TPP3 pairs
    tpp3_pairs = (test_df['task'] == 'TPP3').sum()

    # Print the results
    print(f"\n**Wenn {test_file_name} als Testset verwendet wird:**")
    print(f"  - TPP3-Paare im Testset: {tpp3_pairs}")
    print(f"  - Gesamt Test-Paare: {len(test_df)}")
